In [6]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [17]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 26 13:29:18 2020

@author: Kasia
"""
#####  SAVING FILE  & BOLD TEXT FUNCTIONS  #####

def export_result_to_csv(result_data):
    import os
    cwd = os.getcwd()
    file_name = create_file_name()
    result_data.to_csv(("Zara_Promos_" + file_name + ".csv"), index=False)
    print("\n")
    printmd("Your results are saved in:")
    print(cwd)
    return None

def create_file_name():
    from datetime import datetime
    import time
    t = time.localtime()
    current_time = time.strftime("%b_%d_%H_%M", t)   
    return current_time

def printmd(string):
    display(Markdown('**'+string+'**'))


######  HANDLING EXCEPTIONS FUNCTIONS  ######
from requests.exceptions import SSLError, Timeout, TooManyRedirects, RequestException
from contextlib import closing
from IPython.display import Markdown, display
import warnings

warnings.filterwarnings('ignore')

timeout_glob=0.05
verify_glob=True
allow_glob=False

    
def is_good_response(x):
    content_type = x.headers['Content-Type'].lower()
    return ((x.status_code==200 and content_type is not None and content_type.find('html')>-1), x.status_code)

def log_error(e):
    print(e)
    
def status_handler(status):
    if status<300:
        print('Status code correct!')
    elif status>=400 and status <500:
        print("Request failed on user's side. Check the link, check the server, check whether you have an access")
    else:
        print('Request failed on server side.')
    return None

#####  SCRAPING FUNCTION  #####
    
import requests as r
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd


def scrape_zara_promos (url):
    
    soup=BeautifulSoup(r.get(url).content)
    
    #souping and cleaning of articles names
    table_name = soup.select('.name')
    name = [i.text for i in table_name]

    #souping and basic cleaning of original prices
    table_original_price = soup.select('.line-through')
    original_price = [str(i) for i in table_original_price]

    #souping and basic cleaning of sale purcentage
    table_label = soup.select('.label')
    label = [''.join(i.text.split('OFF')[:1]).strip(' ') for i in table_label]

    #souping cleaning of sale price
    table_sale = soup.select('.sale')
    sale_price = [str(i) for i in table_sale]
    
    
    #additional cleaning of original price and making a DataFrame of 1 column
    original_price2 = [re.findall(r'\d+(?:\,\d+)?',i) for i in original_price]
    op = pd.DataFrame(original_price2, columns = ["Original Price"])
    
    #additional cleaning of sale price and making a DataFrame of 1 column
    sale_price2 = [re.findall(r'\d+(?:\,\d+)?',i) for i in sale_price]
    sp = pd.DataFrame(sale_price2, columns=["Sale Price"])
    
    #creating a DataFrame of 2 columns and concating them to orinal price and sale price columns
    df = pd.DataFrame(list(zip(name, label)), columns =['Name', 'Sale'])
    result = pd.concat([df, op, sp], axis=1, sort=False)
    
    export_result_to_csv(result)
    
        
    return result


#####################
    
def adaptive_scraping(url, timeout=timeout_glob, verify=verify_glob):
    global timeout_glob
    global verify_glob
    global allow_glob
    
    #import schedule
    #import time
    
    #schedule.every().sunday.at('22:23').do(adaptive_scraping)
    #while 1:
        #schedule.run_pending()
        #time.sleep(1)
    
    
    try:
        with closing(r.get(url, stream=True, timeout=timeout_glob, verify=verify_glob, allow_redirects=allow_glob)) as resp:
            test1=is_good_response(resp)
            if test1[0]:
                return scrape_zara_promos(url)
            else:
                return status_handler(test[1])
    except Timeout as e:
        log_error(f'Error during requests to {url}: {str(e)}')
        printmd('We can increase response waiting time. Do you want to continue?')
        ans=input('\n[Yes/No]\t')
        if ans.lower()=='yes':
            timeout_glob*=2
            return scrape_zara_promos(url)
        else:
            return None
    except SSLError as e:
        log_error(f'Error during requests to {url}: {str(e)}')
        printmd('We can skip verification of SSL. Do you want to continue?')
        ans=input('\n[Yes/No]\t')
        if ans.lower()=='yes':
            verify_glob=False
            return scrape_zara_promosfunc(url)
        else:
            return None
        
    except TooManyRedirects as e:
        log_error(f'Error during requests to {url}: {str(e)}')
        printmd('We can increase the number of allowed redirects. Do you want to continue?')
        ans=input('\n[Yes/No]\t')
        if ans.lower()=='yes':
            allow_glob=True
            return scrape_zara_promos(url)
        else:
            return None

        
        

In [18]:
adaptive_scraping('https://www.zara.com/fr/fr/femme-event-l1053.html?v1=1314514', timeout=timeout_glob, verify=verify_glob)    

**Your results are saved in:**

C:\Users\Kasia\TEST\data-prework\3. -Moja prep nauka\Zara Promo Web Scrapping


,Name,Sale,Original Price,Sale Price
0,TOP AJOURÉ,40%,"39,95","23,97"
1,BLOUSE À COL CLAUDINE,40%,"29,95","17,97"
2,BLOUSE À NŒUD ET VOLANTS,40%,"39,95","23,97"
3,CHEMISE EN JACQUARD,40%,"39,95","23,97"
4,TOP AJOURÉ,40%,"39,95","23,97"
...,...,...,...,...
331,SANDALES PLATES EN CUIR AVEC SEMELLE BOIS,40%,"49,95","29,97"
332,ESPADRILLES EN JUTE NATURELLE NOUÉES,40%,"49,95","29,97"
333,CHAUSSURES À TALONS HAUTS EN MÉTHACRYLATE,40%,"39,95","23,97"
334,CHAUSSURES À TALONS EN VINYLE,40%,"39,95","23,97"
